In [186]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import datetime as dt
import math as mt
from pymongo import MongoClient as mc
import IPython
import IPython.display
from sklearn.decomposition import TruncatedSVD 
from scipy.spatial import distance
import scipy.stats
# from modules.models import *
import modules.new_utils as nu
from tqdm import trange

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

[autoreload of modules.new_utils failed: Traceback (most recent call last):
  File "/Users/formegusto/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/formegusto/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/formegusto/opt/anaconda3/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/formegusto/opt/anaconda3/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/formegusto/Desktop/idoformegusto/development/research/contract-recommendation-system/prototype/notebook/modules/new_utils/

In [4]:
'''
title
:
"시뮬레이션 입니다."
dataName
:
"original-data-1644541675739.xlsx"
minPer
:
20
maxPer
:
80
step
:
"similarity-analysis"
createdAt
:
"2022-02-11T10:07:56"
updatedAt
:
"2022-02-11T10:17:39"
'''

# 1. read datas.xlsx
xlsx = nu.load_excel("datas/datas.xlsx")

In [78]:
mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)
db = client.crs_test
col = db.result

In [79]:
init_data = {
    "title":"시뮬레이션 입니다.",
    "dataName":"original-data-1644541675739.xlsx",
    "minPer": 20,
    "maxPer": 80,
    "step":"similarity-analysis",
    "createdAt":"2022-02-11T10:07:56",
    "updatedAt":"2022-02-11T10:17:39"
}
init_result = col.insert_one(init_data)
pid = init_result.inserted_id

print(pid)

6205e286c22e94d548b416d2


In [196]:
# 2. data pre-processing
p, m, d = nu.data_preprocessing(xlsx, db_processing=True)

if d != None:
    col.update_one({"_id": pid},{
        "$set":{
            "dpp": d
        }
    })

/Users/formegusto/Desktop/idoformegusto/development/research/contract-recommendation-system/prototype/notebook/modules/new_utils/data_preprocessing.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  datas_df[household_name] = household_df[col][3:].to_list()
/Users/formegusto/Desktop/idoformegusto/development/research/contract-recommendation-system/prototype/notebook/modules/new_utils/data_preprocessing.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  month_usage_df[name] = [


In [197]:
# 3. bill - calc
min_per = 10
max_per = 80
bc_result = nu.bill_calc(m, p, min_per, max_per)

1 월 계산 진행 합니다.
가구 수: 398 / 평균: 282 / 표준편차: 107 / 변동계수: 38
2 월 계산 진행 합니다.
가구 수: 398 / 평균: 254 / 표준편차: 91 / 변동계수: 36
3 월 계산 진행 합니다.
가구 수: 398 / 평균: 266 / 표준편차: 91 / 변동계수: 34
4 월 계산 진행 합니다.
가구 수: 398 / 평균: 249 / 표준편차: 86 / 변동계수: 35
5 월 계산 진행 합니다.
가구 수: 398 / 평균: 183 / 표준편차: 126 / 변동계수: 69
6 월 계산 진행 합니다.
가구 수: 398 / 평균: 226 / 표준편차: 84 / 변동계수: 37
7 월 계산 진행 합니다.
가구 수: 398 / 평균: 341 / 표준편차: 133 / 변동계수: 39
8 월 계산 진행 합니다.
가구 수: 398 / 평균: 349 / 표준편차: 140 / 변동계수: 40
9 월 계산 진행 합니다.
가구 수: 398 / 평균: 235 / 표준편차: 85 / 변동계수: 36
10 월 계산 진행 합니다.
가구 수: 398 / 평균: 246 / 표준편차: 88 / 변동계수: 36
11 월 계산 진행 합니다.
가구 수: 398 / 평균: 255 / 표준편차: 94 / 변동계수: 37
12 월 계산 진행 합니다.
가구 수: 398 / 평균: 284 / 표준편차: 111 / 변동계수: 39


In [198]:
# 4. normal_analysis
# 일반분석
# - 각 월 별 유리 가구 수, 아파트 전체 사용요금, 공동설비사용요금, 손실율에서 종합계약이 언제부터 유리해지는 지 확인할 수 있다.
na_result = nu.normal_analysis(bc_result)

In [201]:
# 추천이 들어가는 곳
# 5. mean_analysis
# 평균치를 기준으로 사용량을 재구성하여 일반분석 진행
mean_result = nu.mean_analysis(m, p, min_per, max_per, db_processing=True)

col.update_one({"_id": pid},{
    "$set":{
        "recoPercentage": mean_result['changePer']['positiveCount'],
        "meanAnalysis": mean_result
    }
})

1 월 계산 진행 합니다.
가구 수: 398 / 평균: 264 / 표준편차: 89 / 변동계수: 34


In [202]:
# 추천이 들어가는 곳
# 6. similarity_analysis
# 유사도 높은 패턴들을 기준으로 사용량을 재구성하여 일반분석 진행
anal_result = nu.similarity_analysis(m, p, min_per, max_per, db_processing=True)

col.update_one({"_id": pid},{
    "$set":{
        "simAnalysis": anal_result
    }
})

1 월 계산 진행 합니다.
가구 수: 398 / 평균: 255 / 표준편차: 87 / 변동계수: 34
